In [1]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [2]:
df = pd.read_csv('etc.csv',parse_dates=True, index_col=('date'))
df.dropna(inplace=True)
df.rename(columns={'date':'Date','PriceUSD':'Close'}, inplace=True)
futures = df.tail()

df1 = df[['High','FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
futures1 = df1.tail()

df2 = df[['High','FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
futures2 = df2.tail()


In [3]:
# df.Close.head()

In [4]:
# df = df[:-3]
df.High.tail()

date
2019-11-06    5.17
2019-11-07    5.30
2019-11-08    5.29
2019-11-09    5.05
2019-11-10    5.09
Name: High, dtype: float64

In [5]:
#Model0
X=df[:-5].copy()
X.head()
y = df["High"][5:].values.reshape(-1, 1)
y[:5]
#Model1
X1=df1[:-5].copy()
X1=X1[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
y1 = df1["High"][5:].values.reshape(-1, 1)
y1[:5]
#Model2
X2=df2[:-5].copy()
X2=X2[['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
y2 = df2["High"][5:].values.reshape(-1, 1)
y2[:5]

array([[14.41],
       [14.14],
       [14.69],
       [15.11],
       [15.48]])

In [6]:
X.tail()

,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMrktCurUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,...,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d,High,Low
date,,,,,,,,,,,,,,,,,,,,,
2019-11-01,463640,6657,10232466,1537.098693,560790095.2,1.240000e+14,0.000177,0.000864,0.000021,0.000102,...,154.557973,0.999252,4.875223,1.111490e+06,5422821.047,0.048307,0.037313,0.040112,4.96,4.82
2019-11-02,480938,6623,10109872,1526.479239,566530039.0,1.290000e+14,0.000175,0.000864,0.000021,0.000103,...,93.351351,0.998957,4.922534,6.346154e+05,3127176.919,0.048310,0.037269,0.040114,5.02,4.85
2019-11-03,487123,6626,10060597,1518.351494,561770693.1,1.260000e+14,0.000162,0.000791,0.000021,0.000103,...,75.767189,0.998896,4.879751,5.223359e+05,2551687.399,0.048313,0.037288,0.040113,5.01,4.85
2019-11-04,456285,6492,10046189,1547.472120,570484726.5,1.250000e+14,0.000184,0.000911,0.000021,0.000104,...,94.442132,0.999549,4.957563,6.564563e+05,3255892.489,0.048322,0.037364,0.039646,4.99,4.88
2019-11-05,450544,6560,10060327,1533.586433,577760237.2,1.230000e+14,0.000179,0.000898,0.000021,0.000105,...,120.393431,0.999342,5.018602,8.351223e+05,4193905.185,0.048304,0.036883,0.039343,5.09,4.90


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state=8)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=8)

In [8]:
# Scale the training data
scaler = StandardScaler()
scaler1=StandardScaler()
scaler2=StandardScaler()
#Model0
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#model1
X_scaler1 = scaler1.fit(X_train1)
X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)
#model2
X_scaler2 = scaler2.fit(X_train2)
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

In [9]:
# Choose learning rate
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=learning_rate,
        max_features=2,
        max_depth=3,
        random_state=0)
    model.fit(X_train_scaled,y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        model.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        model.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.976
Accuracy score (validation): 0.963

Learning rate:  0.1
Accuracy score (training): 0.988
Accuracy score (validation): 0.965

Learning rate:  0.25
Accuracy score (training): 0.996
Accuracy score (validation): 0.961

Learning rate:  0.5
Accuracy score (training): 0.999
Accuracy score (validation): 0.959

Learning rate:  0.75
Accuracy score (training): 0.999
Accuracy score (validation): 0.919

Learning rate:  1
Accuracy score (training): 1.000
Accuracy score (validation): 0.884



In [10]:
# Create GradientBoostingClassifier model
#Changed from .75 to .25 11/8
#on 11/8 .75 gave an accuracy score of .929
#model0
model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.15,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model.score(
        X_train_scaled,
        y_train)))
print("Accuracy score (validation): {0:.3f}".format(
    model.score(
        X_test_scaled,
        y_test)))

Accuracy score (training): 1.000
Accuracy score (validation): 0.977


In [11]:
#model1 - [['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
# Create GradientBoostingClassifier model
#Changed from 1 to .05 11/8
#on 11/8 1 gave an accuracy score of .911
model1 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.05,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model1.fit(X_train_scaled1,y_train1.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model1.score(
        X_train_scaled1,
        y_train1)))
print("Accuracy score (validation): {0:.3f}".format(
    model1.score(
        X_test_scaled1,
        y_test1)))

Accuracy score (training): 0.994
Accuracy score (validation): 0.969


In [12]:
#model2 - [['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
#Changed from .1 to .05 11/8
#on 11/8 .1 gave an accuracy score of .938
model2 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.05,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model2.fit(X_train_scaled2,y_train2.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model2.score(
        X_train_scaled2,
        y_train2)))
print("Accuracy score (validation): {0:.3f}".format(
    model2.score(
        X_test_scaled2,
        y_test2)))

Accuracy score (training): 0.995
Accuracy score (validation): 0.967


In [13]:
# Make predictions
#model0
predictions = model.predict(X_test_scaled)
y_test = y_test.ravel()
actual = pd.Series(y_test)

#model1
predictions1 = model1.predict(X_test_scaled1)
y_test1 = y_test1.ravel()
actual1 = pd.Series(y_test1)
#model2
predictions2 = model2.predict(X_test_scaled2)
y_test2 = y_test2.ravel()
actual2 = pd.Series(y_test2)

In [14]:
pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual': actual}).head()

,Model0,Model1,Model2,Actual
0,4.726978,4.627288,4.474862,5.00
1,5.426981,4.942591,4.926511,5.45
2,9.531761,9.706640,10.556085,9.46
3,9.425163,8.822104,8.373989,6.48
4,36.402244,42.204313,41.501677,39.05


In [15]:
# X.tail()


In [16]:
X_future = X_scaler.transform(X.tail(5))
predictions = model.predict(X_future)

X_future1 = X_scaler1.transform(X1.tail(5))
predictions1 = model1.predict(X_future1)

X_future2 = X_scaler2.transform(X2.tail(5))
predictions2 = model2.predict(X_future2)

In [17]:
actual=futures['High'].ravel()
dates=futures.reset_index()
dates=dates['date'].tail(5).ravel()
# Model1
actual1=futures1['High'].ravel()
dates1=futures1.reset_index()
dates1=dates1['date'].tail(5).ravel()
#Model2
actual2=futures2['High'].ravel()
dates2=futures2.reset_index()
dates2=dates2['date'].tail(5).ravel()

In [18]:
test_final=pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual':actual, 'Date':dates})

In [19]:
test_final

,Model0,Model1,Model2,Actual,Date
0,5.172681,5.165917,5.152373,5.17,2019-11-06
1,5.298601,5.165917,5.362265,5.30,2019-11-07
2,5.265557,5.165917,5.171648,5.29,2019-11-08
3,4.978269,5.165917,5.074295,5.05,2019-11-09
4,5.134767,5.163856,5.450087,5.09,2019-11-10


In [20]:
#Model0
real_future = X_scaler.transform(df.tail(5))
predictions = model.predict(real_future)
# #Model1
# real_future1 = X_scaler1.transform(df1.tail(5))
# predictions1 = model1.predict(real_future1)
# #Model2
# real_future2 = X_scaler2.transform(df2.tail().drop('Close',axis=1))
# predictions2 = model2.predict(real_future2)

Based on my predictions today, (Nov 7th)- I should sell my stock tomorrow (Nov 8th)


In [21]:
datelist = pd.date_range((pd.datetime.today()), periods=5).tolist()

# datelist = pd.date_range((pd.datetime.today()+timedelta(days=-3)), periods=5).tolist()
future_dates=[]
for date in datelist:
    future_dates.append(date.strftime("%m-%d-%Y"))

In [22]:
final=pd.DataFrame({'Date':future_dates,'Model0':predictions})
# final['Avg']=(final.Model0+final.Model1+final.Model2) / 3

In [23]:
final

,Date,Model0
0,11-11-2019,5.223585
1,11-12-2019,5.245388
2,11-13-2019,4.922950
3,11-14-2019,4.840305
4,11-15-2019,5.089093


In [24]:
# final.to_csv('11_11.csv')


In [25]:
cols = final.columns.tolist()
cols = cols[-1:] + cols[:-1]
final=final[cols]


In [26]:
final

,Model0,Date
0,5.223585,11-11-2019
1,5.245388,11-12-2019
2,4.922950,11-13-2019
3,4.840305,11-14-2019
4,5.089093,11-15-2019
